# 11. WORKING WITH GEOSPATIAL DATA 
(_35min_)

## 11.1 Module introduction

- Storing geospatial data in `geoJSON` format
- Querying geospatial data

---

## 11.2 Adding geoJSON data

1. [GeoJSON Objects](https://www.mongodb.com/docs/manual/reference/geojson/)

> MongoDB supports the GeoJSON object types listed on this page.
> 
> 
> To specify GeoJSON data, use an embedded document with:
> 
> - a field named `type` that specifies the GeoJSON object type, and
> - a field named `coordinates` that specifies the object's coordinates.
> 
> ```jsx
> <field>: {type: <GeoJSON type> , coordinates: <coordinates> }
> ```
> 
> **Important**
> 
> If specifying latitude and longitude coordinates, list the **longitude** first, and then **latitude**.
> 
> - Valid longitude values are between -`180` and `180`, both inclusive.
> - Valid latitude values are between -`90` and `90`, both inclusive.
> 
> MongoDB geospatial queries on GeoJSON objects calculate on a sphere; MongoDB uses the [**WGS84**](https://www.mongodb.com/docs/manual/reference/glossary/#std-term-WGS84) reference system for geospatial queries on GeoJSON objects.
>

- **Exemplo de inserção de documento**:

In [ ]:
// use awesomeplaces
 
// California Academy of Sciences -> @37.7698688,-122.4686696
db.places.insertOne(
 	{ name: "California Academy of Sciences", 
  		location:
 			{ type: "Point", 
 				coordinates: [ -122.4686696, 37.7698688 ]
 			}
 	} ) //  ObjectId('67f7a9bee66a059843b71236'

---

## 11.3 Running Geo queries

> Specifies a point for which a [**geospatial**](https://www.mongodb.com/docs/manual/reference/glossary/#std-term-geospatial) query returns the documents from nearest to farthest. The [**`$near`**](https://www.mongodb.com/docs/manual/reference/operator/query/near/#mongodb-query-op.-near) operator can specify either a [**GeoJSON**](https://www.mongodb.com/docs/manual/reference/glossary/#std-term-GeoJSON) point or legacy coordinate point.
> 
> 
> [**`$near`**](https://www.mongodb.com/docs/manual/reference/operator/query/near/#mongodb-query-op.-near) requires a geospatial index:
> 
> - [**2dsphere**](https://www.mongodb.com/docs/manual/core/indexes/index-types/geospatial/2dsphere/#std-label-2dsphere-index) index if specifying a [**GeoJSON**](https://www.mongodb.com/docs/manual/reference/glossary/#std-term-GeoJSON) point.
> - [**2d**](https://www.mongodb.com/docs/manual/core/indexes/index-types/geospatial/2d/#std-label-2d-index) index if specifying a point using legacy coordinates.
> 
> To specify a [**GeoJSON**](https://www.mongodb.com/docs/manual/reference/glossary/#std-term-GeoJSON) point, [**`$near`**](https://www.mongodb.com/docs/manual/reference/operator/query/near/#mongodb-query-op.-near) operator requires a [**2dsphere**](https://www.mongodb.com/docs/manual/core/indexes/index-types/geospatial/2dsphere/#std-label-2dsphere-index) index and has the following syntax:
> 
> ``` javascript
> { <location field>: 
> 	{ $near:
> 		{ $geometry: 
> 			{ type: "Point", 
> 				coordinates: [ <longitude> , <latitude> ]
> 			},
> 		$maxDistance: <distance in meters>,
> 		$minDistance: <distance in meters>
> 		}
> 	}
> }
> ```
>

In [ ]:
// fake user location: @37.7706554,-122.470751
db.places.find( { location: { $near: { $geometry: { type: "Point", coordinates: [ -122.470751, 37.7706554 ] } } } } )

- A tentativa de inserção acima retornou o erro abaixo:

In [ ]:
Uncaught:
MongoServerError[NoQueryExecutionPlans]: error processing query: ns=awesomeplaces.placesTree: GEONEAR  field=location maxdist=1.79769e+308 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query

---

## 11.4 Adding a geospatial index to track de distance

1. [Geospatial Indexes](https://www.mongodb.com/docs/manual/core/indexes/index-types/index-geospatial/#geospatial-indexes)
2. [Geospatial Indexes Restrictions](https://www.mongodb.com/docs/manual/core/indexes/index-types/geospatial/restrictions/#geospatial-index-restrictions)
3. [Geospatial Indexes: `$maxDistance`](https://www.mongodb.com/docs/manual/reference/operator/query/maxDistance/#-maxdistance)
4. [Geospatial Indexes: `$minDistance`](https://www.mongodb.com/docs/manual/reference/operator/query/minDistance/#-mindistance)

- Após a criação do index, a inserção de um documento com conteúdo geoespacial ocorre com sucesso.

In [ ]:
db.places.createIndex( { location: "2dsphere" } ) // location_2dsphere
db.places.find( { location: { $near: { $geometry: { type: "Point", coordinates: [ -122.470751, 37.7706554 ] } } } } ) // now it works

// fake user location: @37.7706554,-122.470751
db.places.find( { location: { $near: { $geometry: { type: "Point", coordinates: [ -122.470751, 37.7706554 ] }, $maxDistance: 500, $minDistance: 10 } } } )

---

## 11.5 Adding additional locations

In [ ]:
// Conservatory of Flowers: @37.7715904,-122.466894
db.places.insertOne(
	{ name: "Conservatory of Flowers", 
		location:
			{ type: "Point", 
				coordinates: [ -122.466894, 37.7715904 ]
			}
	} )

// Golden Gate Park - East: @37.7710052,-122.464244
db.places.insertOne(
	{ name: "Golden Gate Park - East", 
		location:
			{ type: "Point", 
				coordinates: [ -122.464244, 37.7710052 ]
			}
	} )
    
// Universidade de São Francisco: @37.7756017,-122.4574848
db.places.insertOne(
	{ name: "University of San Francisco", 
		location:
			{ type: "Point", 
				coordinates: [ -122.4574848, 37.7756017 ]
			}
	} )

---

## 11.6 Finding places inside a certain AREA

1. [`$geoWithin`](https://www.mongodb.com/docs/manual/reference/operator/query/geoWithin/#-geowithin)
2. [Query for Locations Bound by a Polygon](https://www.mongodb.com/docs/manual/core/indexes/index-types/geospatial/2dsphere/query/geojson-bound-by-polygon/#query-for-locations-bound-by-a-polygon)

- Query para verificar se determinada coordenada se encontra dentro de uma área específica
    - Área: caixa com extremidades pontuadas em amarelo
    - Locais dentro da área: pontuados em verde
    - Local fora da área: pontuado em vermelho

<img src="imgs\s11\s11-1.png" width=800 height=300 >

- A query abaixo deve retornar somente 3 locais dos 4 existentes na coleção "places", pois somente 3 locais estão dentro da área delimitada pelo poligono passado
- Note que, ao repassar as coordenadas, o ponto "p1" é repetido no final. Isto é necessário, pois um polígono espera o ponto inicial para saber que todos os valores já foram recebidos.

In [ ]:
// 1st: 37.77469, -122.45475
// 2nd: 37.76643, -122.45292
// 3rd: 37.76421, -122.51021
// 4th: 37.77134, -122.51088

const p1 = [ -122.45475, 37.77469 ]
const p2 = [ -122.45292, 37.76643 ]
const p3 = [ -122.51021, 37.76421 ]
const p4 = [ -122.51088, 37.77134 ]

db.places.find( { location: { $geoWithin: { $geometry: {type: "Polygon", coordinates: [ [ p1, p2, p3, p4, p1 ]]}}}})

---

## 11.7 Finding out if a user is inside a specific AREA

- [Query for Locations that Intersect a GeoJSON Object](https://www.mongodb.com/docs/manual/core/indexes/index-types/geospatial/2dsphere/query/intersections-of-geojson-objects/#query-for-locations-that-intersect-a-geojson-object)
- [`$geoIntersects`](https://www.mongodb.com/docs/manual/reference/operator/query/geoIntersects/#-geointersects)

<img src="imgs\s11\s11-2.png" width=800 height=300 >

- Vamos criar uma coleção para armazenarmos as "áreas" e inserir a área do "Golden Gate Park" nela

In [ ]:
// user location inside the area: 37.76869, -122.49166

db.areas.insertOne( { name: "Golden Gate Park", area: {type: "Polygon", coordinates: [ [ p1, p2, p3, p4, p1 ]]} })

- Agora vamos executar uma query para verificar se o usuário está dentro da área
- Primeiro, é necessário criar um index

In [ ]:
db.areas.createIndex( { area: "2dsphere" }) // area_2dsphere

db.areas.find( { area: { $geoIntersects: { $geometry: { type: "Point", coordinates: [ -122.49166, 37.76869 ]}}}})

<img src="imgs\s11\s11-3.png" width=800 height=300 >

In [ ]:
// user location outside the area: 37.75736, -122.48306

db.areas.find( { area: { $geoIntersects: { $geometry: { type: "Point", coordinates: [ -122.48306, 37.75736 ]}}}})

- Como o usuário está fora da única área que temos cadastrada na coleção, a query não retorna nada

---

## 11.8 Finding places within a certain RADIUS

1. [Query for Locations within a Circle on a Sphere](https://www.mongodb.com/docs/manual/core/indexes/index-types/geospatial/2dsphere/query/points-within-circle-on-sphere/#query-for-locations-within-a-circle-on-a-sphere)
2. [`$centerSphere`](https://www.mongodb.com/docs/manual/reference/operator/query/centerSphere/#-centersphere)

> Defines a circle for a geospatial query that uses spherical geometry. The query returns documents that are within the bounds of the circle.
>
>You can use the $centerSphere operator on both GeoJSON objects and legacy coordinate pairs.
> 
> To use `$centerSphere`, specify an array that contains:
> The grid coordinates of the circle's center point, and
> 
> The circle's radius measured in radians. To calculate radians, see [Convert Distance to Radians for Spherical Operators](https://www.mongodb.com/docs/manual/core/indexes/index-types/geospatial/2d/calculate-distances/#std-label-calculate-distance-spherical-geometry)

<img src="imgs\s11\s11-4.png" width=550 height=350 >

In [ ]:
// Angler's Lodge - Golden Gate Park: 37.76804, -122.49665
// Bison Paddock: 37.76969, -122.49823
// Bercut Equitation Field: 37.76647, -122.50129
// Radius-User: 37.76865, -122.49908

db.places.insertMany( [
	{ name: "Angler's Lodge - Golden Gate Park", 
		location:
			{ type: "Point", 
				coordinates: [ -122.49665, 37.76804 ]
			}
	},
    { name: "Bison Paddock", 
		location:
			{ type: "Point", 
				coordinates: [ -122.49823, 37.76969 ]
			}
	},
    { name: "Bercut Equitation Field", 
		location:
			{ type: "Point", 
				coordinates: [ -122.50129, 37.76647 ]
			}
	}
] )

- A query abaixo retorna somente 2 resultados:

``` javascript
[
  {
    _id: ObjectId('67f81047e66a059843b7123b'),
    name: "Angler's Lodge - Golden Gate Park",
    location: { type: 'Point', coordinates: [ -122.49665, 37.76804 ] }
  },
  {
    _id: ObjectId('67f81047e66a059843b7123c'),
    name: 'Bison Paddock',
    location: { type: 'Point', coordinates: [ -122.49823, 37.76969 ] }
  }
]
```

In [ ]:
// user location for radius activity: 37.76865, -122.49908

db.places.find( { location: { $geoWithin: { $centerSphere: [ [ -122.49908, 37.76865 ], 0.3 / 6378.1 ]}}}) // 0.3/6378.1 = 300 metros

- Os outros locais existentes na coleção estão há mais de 300mts do usuário definido

<img src="imgs\s11\s11-5.png" width=400 height=250 >

---

## 11.9 Assignment: Time to Practice - Geospatial Data

1. Pick 3 points on Google Maps and store them in a collection
2. Pick a point and find the nearest points within a min and max distance
3. Pick an area and see which points (that are stored in your collection) it contains
4. Store at least one area in a different collection
5. Pick a point and find out which areas in your collection contain that point

### 11.9.1. Pick 3 points on Google Maps and store them in a collection

In [ ]:
// FIT: -23.42344, -47.36487
// Flex: -23.42964, -47.36755
// ABB: -23.42625, -47.36313

// use assign9

db.places.createIndex( {location: "2dsphere"} ) //location_2dsphere

db.places.insertMany( [ 
    { name: "FIT", location: { type: "Point", coordinates: [ -47.36487, -23.42344 ] } },
    { name: "Flex", location: { type: "Point", coordinates: [ -47.36755, -23.42964 ] } },
    { name: "ABB", location: { type: "Point", coordinates: [ -47.36313, -23.42625 ] } }
] )

<img src="imgs\s11\s11-7.png" width=600 height=300 >

### 11.9.2 Pick a point and find the nearest points within a min and max distance


In [ ]:
// Nextracker: -23.42801, -47.3644

db.places.find( 
    { location: 
        { $near: 
            { $geometry: 
                { type: "Point", coordinates: [ -47.3644, -23.42801 ] },
                $maxDistance: 300,
                $minDistance: 50    
            }
        }
    }
)

- Resultado da query acima:
    - 1 retorno, pois somente a localização "ABB" está a mais de 50mt e menos de 300mt da localização "Nextracker"

In [ ]:
[
    {
      _id: ObjectId('67f902dedd7a1b5c78b71238'),
      name: 'ABB',
      location: { type: 'Point', coordinates: [ -47.36313, -23.42625 ] }
    }
  ]

### 11.9.3. Pick an area and see which points (that are stored in your collection) it contains

<img src="imgs\s11\s11-8.png" width=600 height=300 >

In [ ]:
// Nextracker: -23.42801, -47.3644
db.places.insertOne( { name: "Nextracker", location: { type: "Point", coordinates: [ -47.3644, -23.42801 ] } } )

// p1: -23.4236, -47.36357
// p2: -23.42636, -47.36149
// p3: -23.43067, -47.36784
// p4: -23.42847, -47.37084

const p1 = [-47.36357, -23.4236]
const p2 = [-47.36149, -23.42636]
const p3 = [-47.36784, -23.43067]
const p4 = [-47.37084, -23.42847]

db.places.find( { location: { $geoWithin: { $geometry: { type: "Polygon", coordinates: [ [ p1, p2, p3, p4, p1 ] ] } } } } )

- Resultado da query acima:
    - 3 retornos, pois somente as localizações "ABB", "Flex" e "Nextracker" estão dentro da área do poligono repassado
    - A localização "FIT" está claramente fora da área

In [ ]:
[
  {
    _id: ObjectId('67f902dedd7a1b5c78b71237'),
    name: 'Flex',
    location: { type: 'Point', coordinates: [ -47.36755, -23.42964 ] }
  },
  {
    _id: ObjectId('67f902dedd7a1b5c78b71238'),
    name: 'ABB',
    location: { type: 'Point', coordinates: [ -47.36313, -23.42625 ] }
  },
  {
    _id: ObjectId('67f90991dd7a1b5c78b71239'),
    name: 'Nextracker',
    location: { type: 'Point', coordinates: [ -47.3644, -23.42801 ] }
  }
]

### 11.9.4 Store at least one area in a different collection


In [ ]:
// p1: -23.4236, -47.36357
// p2: -23.42636, -47.36149
// p3: -23.43067, -47.36784
// p4: -23.42847, -47.37084

const p1 = [-47.36357, -23.4236]
const p2 = [-47.36149, -23.42636]
const p3 = [-47.36784, -23.43067]
const p4 = [-47.37084, -23.42847]

db.areas.createIndex( { area: "2dsphere" } ) // area_2dsphere

db.areas.insertOne( { name: "Flex Area", location: { type: "Polygon", coordinates: [ [ p1, p2, p3, p4, p1 ]]}})

### 11.9.5 Pick a point and find out which areas in your collection contain that point

<img src="imgs\s11\s11-9.png" width=500 height=300 >

In [ ]:
// newPoint: -23.42695, -47.36696

db.areas.find( { location: { $geoIntersects: { $geometry: { type: "Point", coordinates: [ -47.36696, -23.42695 ]}}}})

- Resultado da query acima:
    - 1 retorno, pois a localização do "newPoint" está dentro da área existente na coleção "areas"
    - Se o ponto estivesse fora dessa única área, o retorno seria vazio
    - Se o ponto estivesse dentro de mais de uma área cadastrada, todas as áreas dentro da localização seriam retornadas

In [ ]:
[
    {
      _id: ObjectId('67f90c1fdd7a1b5c78b7123a'),
      name: 'Flex Area',
      location: {
        type: 'Polygon',
        coordinates: [
          [
            [ -47.36357, -23.4236 ],
            [ -47.36149, -23.42636 ],
            [ -47.36784, -23.43067 ],
            [ -47.37084, -23.42847 ],
            [ -47.36357, -23.4236 ]
          ]
        ]
      }
    }
  ]

---

## 11.10 Wrap up

<img src="imgs\s11\s11-6.png" width=800 height=400 >

---

## 11.11 Useful Resources & Links

>Helpful Articles/ Docs:
>
>- Official Geospatial Docs: https://docs.mongodb.com/manual/geospatial-queries/
>- Geospatial Query Operators: https://docs.mongodb.com/manual/reference/operator/query-geospatial/